In [16]:
from bonsai.data_loaders import load_data
from bonsai.net import Net
from bonsai.trainers import *
from bonsai.helpers import *

import pandas as pd
pd.set_option("display.max_rows", 150)

In [2]:
nas_schedule = {'learn_phase': 16,
                'prune_phase': 16,
                'prune_interval': 4}
hypers = {
    'gpu_space': 8.25,
    'dataset': 'CIFAR10',
    'classes': 10,
    'batch_size': 64,
    'scale': 5,
    'nodes': 4,
    'patterns': [['r','n','n','n','na']],
    'half': False,
    'multiplier': 1,
    'lr_schedule':
        {'lr_max': .01,
         'T': 1},
    'drop_prob': .25,
    'prune_rate': {'edge': .5, 'input': .5}
}
data, dim = load_data(hypers['batch_size'], hypers['dataset'])
hypers['num_patterns'] = 1

model = Net(dim=dim,
            classes=hypers['classes'],
            scale=hypers['scale'],
            num_patterns=hypers['num_patterns'],
            patterns=hypers['patterns'],
            nodes=hypers['nodes'],
            random_ops={'e_c': .25, 'i_c': 1},
            drop_prob=hypers['drop_prob'],
            lr_schedule=hypers['lr_schedule'],
            prune=False)
model.data = data

In [3]:
optimizer = optim.SGD(model.parameters(), lr=model.lr_scheduler.lr, momentum=.9, weight_decay=3e-4)
criterion = nn.CrossEntropyLoss()
model.cuda()
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    train(model,
          torch.device("cuda"),
          criterion=criterion,
          optimizer=optimizer,
          epoch=0,
          kill_at=50)

12/10/2019 03:01 PM
Init: 22.00MiB
0: 826.00MiB
1: 1.44GiB
2: 2.08GiB
3: 2.71GiB
4: 3.36GiB
GP: 3.36GiB
Classifier: 3.36GiB
Train Epoch: 0   [3264  /50000  (7%)]	Per Epoch: 3m,17s , Alloc: 3.36GiB  , Data T: 1.038 , Op T: 13.855
Train Corrects: Top-1: 1.19%, 15.50s


In [4]:
tbl = prof.table(row_limit=None)
rows = tbl.split("\n")
print(len(rows))

747333


In [5]:
rows[-3:]

['Self CPU time total: 10.578s', 'CUDA time total: 21.086s', '']

In [6]:
def white_split(str):
    return [x.strip() for x in str.split("  ") if x.strip()]

header = white_split(rows[1])

In [7]:
table_rows = []
for row in rows[3:-4]:
    fields = white_split(row)
    table_rows.append({header[i]:fields[i] for i in range(len(fields))})
df = pd.DataFrame(table_rows)
for field in ['CPU time avg','CPU total','CUDA time avg','CUDA total','Self CPU total']:
    df[field]=df[field].apply(lambda x: float(x[:-2])*1e-6)
df['Number of Calls']=df['Number of Calls'].apply(int)
df

,CPU time avg,CPU total,CPU total %,CUDA time avg,CUDA total,CUDA total %,Input Shapes,Name,Number of Calls,Self CPU total,Self CPU total %
0,0.000018,0.000018,0.00%,0.000015,0.000015,0.00%,[],random_,1,0.000018,0.00%
1,0.000004,0.000004,0.00%,0.000004,0.000004,0.00%,[],is_floating_point,1,0.000004,0.00%
2,0.000004,0.000004,0.00%,0.000004,0.000004,0.00%,[],is_complex,1,0.000004,0.00%
3,0.000015,0.000015,0.00%,0.000015,0.000015,0.00%,[],item,1,0.000010,0.00%
4,0.000005,0.000005,0.00%,0.000005,0.000005,0.00%,[],_local_scalar_dense,1,0.000005,0.00%
5,0.000032,0.000032,0.00%,0.000029,0.000029,0.00%,[],to,1,0.000024,0.00%
6,0.000007,0.000007,0.00%,0.000008,0.000008,0.00%,[],empty,1,0.000007,0.00%
7,0.000017,0.000017,0.00%,0.000017,0.000017,0.00%,[],div,1,0.000017,0.00%
8,0.000012,0.000012,0.00%,0.000012,0.000012,0.00%,[],clone,1,0.000012,0.00%
9,0.000015,0.000015,0.00%,0.000015,0.000015,0.00%,[],to,1,0.000015,0.00%


In [9]:
df['CUDA total'].sum()

20.706645459

In [17]:
df.groupby('Name')[['CUDA total','Number of Calls']].agg({'CUDA total':['sum','mean'],'Number of Calls':['sum']}).sort_values(by=('CUDA total','sum'),ascending=False)

CUDA total           Number of Calls
                                        sum      mean             sum
Name                                                                 
mul                                1.987779  0.000022           91729
add                                1.394581  0.000027           51740
MulBackward0                       1.290710  0.000045           28574
add_                               1.202789  0.000010          114560
CudnnBatchNormBackward             1.099288  0.000140            7852
cudnn_batch_norm_backward          1.030292  0.000131            7852
ThnnConvDepthwise2DBackward        0.925898  0.000356            2600
batch_norm                         0.914002  0.000116            7852
thnn_conv_depthwise2d_backward     0.898620  0.000346            2600
_batch_norm_impl_index             0.892727  0.000114            7852
cudnn_batch_norm                   0.694080  0.000088            7852
conv2d                             0.534260  0.000127            4212
convolution                        0.523773  0.000124            4212
_convolution                       0.511619  0.000121            4212
sum                                0.505291  0.000072            7020
div                                0.468393  0.000012           39833
CudnnConvolutionBackward           0.444012  0.000275            1612
cudnn_convolution_backward         0.439711  0.000273            1612
DivBackward0                       0.395285  0.000027           14794
thnn_conv_depthwise2d              0.388270  0.000149            2600
mul_                               0.381675  0.000014           26418
thnn_conv_depthwise2d_forward      0.380164  0.000146            2600
MaxPool2DWithIndicesBackward       0.302316  0.000291            1040
max_pool2d_with_indices_backward   0.297348  0.000286            1040
zero_                              0.257380  0.000008           33334
torch::autograd::AccumulateGrad    0.210664  0.000008           26936
FloorBackward                      0.166612  0.000024            6916
max_pool2d                         0.156035  0.000150            1040
SubBackward0                       0.154571  0.000022            6916
max_pool2d_with_indices            0.152988  0.000147            1040
zeros_like                         0.135801  0.000020            6916
AbsBackward                        0.128020  0.000019            6916
contiguous                         0.126986  0.000002           53924
to                                 0.115685  0.000011           10192
abs                                0.111110  0.000016            6916
detach_                            0.096174  0.000004           26470
cudnn_convolution                  0.095148  0.000059            1612
neg                                0.085958  0.000006           13832
floor                              0.083950  0.000012            6916
sub                                0.082742  0.000012            6916
unsqueeze                          0.073177  0.000005           13312
empty                              0.054604  0.000005           11310
ReluBackward0                      0.052382  0.000067             780
threshold_backward                 0.050325  0.000065             780
AddBackward0                       0.047896  0.000002           25116
clone                              0.047295  0.000012            3846
sign                               0.042746  0.000006            6916
slice                              0.036110  0.000005            6708
relu                               0.036015  0.000046             780
sub_                               0.027155  0.000008            3328
div_                               0.021747  0.000007            3328
stack                              0.021425  0.000412              52
view                               0.014481  0.000002            7124
bernoulli                          0.012814  0.000014             910
AddmmBackward                      0.011930

In [18]:
df.groupby('Name')[['CPU total','Number of Calls']].agg({'CPU total':['sum','mean'],'Number of Calls':['sum']}).sort_values(by=('CPU total','sum'),ascending=False)

CPU total           Number of Calls
                                       sum      mean             sum
Name                                                                
mul                               1.618507  0.000018           91729
add_                              1.506604  0.000013          114560
DivBackward0                      1.015001  0.000069           14794
MulBackward0                      0.939106  0.000033           28574
add                               0.879523  0.000017           51740
batch_norm                        0.774938  0.000099            7852
_batch_norm_impl_index            0.702695  0.000089            7852
div                               0.679304  0.000017           39833
torch::autograd::AccumulateGrad   0.602097  0.000022           26936
CudnnBatchNormBackward            0.459319  0.000058            7852
mul_                              0.384906  0.000015           26418
SubBackward0                      0.369064  0.000053            6916
AbsBackward                       0.325322  0.000047            6916
conv2d                            0.321405  0.000076            4212
cudnn_batch_norm                  0.296125  0.000038            7852
FloorBackward                     0.287594  0.000042            6916
zero_                             0.286977  0.000009           33334
convolution                       0.284501  0.000068            4212
cudnn_batch_norm_backward         0.266897  0.000034            7852
contiguous                        0.253641  0.000005           53924
_convolution                      0.248561  0.000059            4212
neg                               0.247755  0.000018           13832
zeros_like                        0.225632  0.000033            6916
sum                               0.191080  0.000027            7020
CudnnConvolutionBackward          0.146954  0.000091            1612
ThnnConvDepthwise2DBackward       0.142235  0.000055            2600
floor                             0.132417  0.000019            6916
cudnn_convolution_backward        0.129451  0.000080            1612
abs                               0.126501  0.000018            6916
to                                0.124371  0.000012           10192
sub                               0.117930  0.000017            6916
AddBackward0                      0.115960  0.000005           25116
sign                              0.107519  0.000016            6916
detach_                           0.096599  0.000004           26470
thnn_conv_depthwise2d_backward    0.090523  0.000035            2600
thnn_conv_depthwise2d             0.084844  0.000033            2600
empty                             0.079546  0.000007           11310
cudnn_convolution                 0.077896  0.000048            1612
unsqueeze                         0.068861  0.000005           13312
thnn_conv_depthwise2d_forward     0.060526  0.000023            2600
view                              0.059492  0.000008            7124
clone                             0.050163  0.000013            3846
max_pool2d                        0.044035  0.000042            1040
MaxPool2DWithIndicesBackward      0.043978  0.000042            1040
bernoulli                         0.039697  0.000044             910
slice                             0.037253  0.000006            6708
max_pool2d_with_indices           0.033272  0.000032            1040
max_pool2d_with_indices_backward  0.032533  0.000031            1040
sub_                              0.026163  0.000008            3328
ReluBackward0                     0.022818  0.000029             780
div_                              0.021912  0.000007            3328
stack                             0.020502  0.000394              52
relu                              0.017788  0.000023             780
threshold_backward                0.014863  0.000019             780
empty_like                        0.013843  0.000015             910
bernoulli_                     